In [11]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
from tensorflow import keras
import tensorflow as tf
import numpy as np
import imutils
import time
import cv2
import os
import smtplib
from email.message import EmailMessage
import time
import threading
import datetime

In [12]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:

        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)

In [13]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [14]:
# load the face mask detector model from disk
recogNet = load_model(r"face_recog_ani_shah_tuna.model")
maskNet = load_model(r"mask_detector_final.model")

In [15]:
def email_alert(subject,body, to):
    msg=EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = to
    user= "cse.setnu.1703170@gmail.com"
    msg['from']= user
    password="wysncrwfnoevdvwl"
    
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user,password)
    server.send_message(msg)
    
    server.quit()


In [16]:
total_count=0
count={"aniket":0,"shahnawaz":0,"tunavi":0}
flag={"aniket":0,"shahnawaz":0,"tunavi":0}
detect={"aniket":0,"shahnawaz":0,"tunavi":0}
to={"aniket":"aniketthakurcse@gmail.com","shahnawaz":"sahmed9957@gmail.com","tunavi":"tunafishyepz@gmail.com"}
subject="COVID-19 Mask Rule Violated by YOU!"
body="Dear Employee,\nYou have been found violating rule by not wearning a mask!\n Please ensure safety protocol is being followed by you."


def decrease_time(rec):
    print("\n"+str(rec) +" is released...\n")
    detect[rec]=0
    flag[rec]=0
#def speak_message(audio):
    #engine.say(audio)
    #engine.runAndWait()

def message(rec):
    if flag[rec]==0 :  
        current_time=datetime.datetime.now().strftime("%H:%M:%S") 
        print("\nmessage sent to " + rec+".... at "+ str(current_time))

        email_alert(subject,body,to[rec])
    
        flag[rec]=1
        count[rec]+=1
        detect[rec]=0
        delay=180                    
        thread_start=threading.Timer(delay,decrease_time,[rec])
        thread_start.start()
        
        
def call_message(rec):
    if flag[rec]==0:
        message(rec)

In [17]:
flag,count

({'aniket': 0, 'shahnawaz': 0, 'tunavi': 0},
 {'aniket': 0, 'shahnawaz': 0, 'tunavi': 0})

In [18]:
def recog_face(frame,faceNet,recogNet):
    (locs, preds) = detect_and_predict_mask(frame, faceNet,recogNet)
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (aniket,shahnawaz,tunavi) = pred

        if aniket>0.80:
            label="aniket"
        elif shahnawaz>0.80:
            label="shahnawaz"
        elif tunavi>0.80:
            label="tunavi"
        else:
            label="Not Found"
        
        if label != "Not Found": 
            detect[label]+=1
            if detect[label]>15:
                call_message(label)
        
            print(label+" is fouded for"+ str(detect[label])+" time")
            
            label = "{}: {:.2f}%".format(label, max(aniket, shahnawaz, tunavi) * 100)
        return label
       

In [19]:
print("[Starting video stream...")
vs = VideoStream("http://192.168.203.115:8080/video").start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=600)

    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        label1 = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label1 == "Mask" else (0, 0, 255)
        
        if label1=="No Mask":
            label_recog=recog_face(frame,faceNet,recogNet)
            cv2.putText(frame, label_recog, (startX, endY +15),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 2)
       
        label1 = "{}: {:.2f}%".format(label1, max(mask, withoutMask) * 100)
    
        cv2.putText(frame, label1, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

        cv2.imshow("Frame", frame)

    if cv2.waitKey(10) == ord('q'):
        break

cv2.destroyAllWindows()
vs.stop()

[Starting video stream...
aniket is fouded for1 time
aniket is fouded for2 time
aniket is fouded for3 time
aniket is fouded for4 time
aniket is fouded for5 time
aniket is fouded for6 time
aniket is fouded for7 time
aniket is fouded for8 time
aniket is fouded for9 time
aniket is fouded for10 time
aniket is fouded for11 time
aniket is fouded for12 time
aniket is fouded for13 time
aniket is fouded for14 time
aniket is fouded for15 time

message sent to aniket.... at 12:33:26
aniket is fouded for0 time
aniket is fouded for1 time
aniket is fouded for2 time
aniket is fouded for3 time
aniket is fouded for4 time
aniket is fouded for5 time


In [20]:
#count


aniket is released...

